In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
import warnings
warnings.filterwarnings(action= 'ignore')
import pickle
# 한글 폰트
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [19]:
data = []
for i in os.listdir('./data'):
    x = pd.read_csv('./data/{}'.format(i),sep = ',')
    data.append(x)
temp = data[2]
data[2] = data[0]
data[0] = temp
del temp

train = data[0]
test = data[1]
submission = data[2]

In [20]:
# 훈련용 데이터
train['일자'] = train['일자'].astype(np.datetime64)

train['년'] = train['일자'].dt.year
train['월'] = train['일자'].dt.month
train['일'] = train['일자'].dt.day

# 테스트 데이터
test['일자'] = test['일자'].astype(np.datetime64)

test['년'] = test['일자'].dt.year
test['월'] = test['일자'].dt.month
test['일'] = test['일자'].dt.day

# column 정렬
train = train[train.columns.tolist()[-3:] + train.columns.tolist()[:-3]]
test = test[test.columns.tolist()[-3:] + test.columns.tolist()[:-3]]

In [21]:
train["식사가능인원"] = (train["본사정원수"] -  train["본사휴가자수"] - train["본사출장자수"] - train["현본사소속재택근무자수"])
train = train[train.columns.tolist()[:-3]+ train.columns.tolist()[-1:] + train.columns.tolist()[-3:-1]]

test["식사가능인원"] = (test["본사정원수"] -  test["본사휴가자수"] - test["본사출장자수"] - test["현본사소속재택근무자수"])

In [22]:
def del_bracket(data):
    import re
    x = []
    for i in data:
        p = re.sub(r'\([^)]*\)',"",i)
        p = re.sub(r'\<[^)]*\>',"",p)
        p = re.findall('[가-힣|a-z|A-Z|ㄱ-ㅎ|*]+',p)
        p = ' '.join(p)
        x.append(p)
        
    return x

def change_column(t,x):
    k = del_bracket(t[x].tolist())
    del t[x]
    t[x] = k

In [23]:
change_column(train,'조식메뉴')
change_column(train,'중식메뉴')
change_column(train,'석식메뉴')
change_column(test,'조식메뉴')
change_column(test,'중식메뉴')
change_column(test,'석식메뉴')

train = train[train.columns[:11].tolist() + train.columns[13:].tolist() + train.columns[11:13].tolist()]

In [24]:
def sauce_punish(data):
    sauce_1 = {}
    for i in range(len(data)):
        x = data[i].split()
        k = ''
        for j in range(len(x)):
            temp = False
            if '*' in x[j]:
                t = x[j].split('*')
                
                x[j] = t[0]
                
                if k == '':
                    k += ' '.join(t[1:])
                else:
                    k += ','
                    k += ' '.join(t[1:])
        
        sauce_1[i] = k
    
        data[i] = ' '.join(x)
        
    sauce = {}
    for i,v in sauce_1.items():
        if v != '':
            sauce[i] = v
            
    return sauce, data

In [25]:
def make_sauce(data):
    temp = data.copy()

    hihi = ['조식','중식','석식']
    for i in hihi:        
        train_bf = temp['{}메뉴'.format(i)].tolist()
        sauce, data = sauce_punish(train_bf)

        temp['{}메뉴'.format(i)] = data
        temp['{}소스 여부'.format(i)] = 0
        temp['{}소스 종류'.format(i)] = 0
        temp['{}소스 여부'.format(i)].iloc[list(sauce.keys())] = 1
        temp['{}소스 종류'.format(i)].iloc[list(sauce.keys())] = list(sauce.values())
    
    return temp

In [26]:
temp_train = make_sauce(train)
temp_t = make_sauce(test)
temp_train = temp_train[temp_train.columns.tolist()[:14] + temp_train.columns.tolist()[16:]  + temp_train.columns.tolist()[14:16]]

# 여기서 부터 입니다

In [27]:
hihi = ['조식','중식','석식']

menu = []
for i in hihi:
    for j in temp_train['{}메뉴'.format(i)]:
        x = j.split(' ')
        for y in x:
            menu.append(y)
            
for i in hihi:
    for j in temp_t['{}메뉴'.format(i)]:
        x = j.split(' ')
        for y in x:
            menu.append(y)
            
menu = list(set(menu))
menu.remove('')
print("menu list contents : {}".format(menu[:10]))
print("menu list length : {}".format(len(menu)))

menu list contents : ['얼큰감자국', '오삼불고기', '버섯들깨나물', '채소전', '쇠고기매운국', '당근크림치즈베이글', '히레카츠', '시리얼샐러드', '토마토계란찜', '올챙이만두국']
menu list length : 2872


In [28]:
def input_menu(x):
    # 만약 저장된 pickle 이 있다면 menu_all 로 열어주기
    try :
        with open('./menu/menu_all.pickle', 'rb') as ma:
            menu_all = pickle.load(ma)
            rice_dict_test = menu_all[0]
            soup_dict_test = menu_all[1]
            vege_dict_test = menu_all[2]
            dessert_dict_test = menu_all[3]
            애매 = menu_all[4]
            
    # 저장된 pickle(메뉴 사전)이 없다면 아래 변수로 초기화
    except:
        rice_dict_test = {
                    "쌀밥류": list(set(list())),
                    "덮밥국밥류": list(set(list())),
                    "비빔밥볶음밥류": list(set(list())),
                    "김초밥류": list(set(list())),
                    "국수류": list(set(list())),
                    "죽류" : list(set(list()))
                    }
        soup_dict_test = {
                    "국탕류": list(set(list())),
                    "찌개류": list(set(list())),
                    "스프류": list(set(lisit()))
                    }
        vege_dict_test = {
                    "김치류": list(set(list())),
                    "샐러드류": list(set(list())),
                    "튀김류": list(set(list())),
                    "전류": list(set(list())),
                    "찜류": list(set(list())),
                    "볶음류": list(set(list())),
                    "조림류": list(set(list())),
                    "구이류": list(set(list())),
                    "무침류": list(set(list())),
                    "장아찌류": list(set(list()))
                    }
        dessert_dict_test = {
                    "음료": list(set(list())),
                    "유제품": list(set(list())),
                    "빵과자류": list(set(list())),
                    "과일류" :  list(set(list())),
                    }
        애매 = {"애매" : list(set(list()))}
        
        menu_all = [rice_dict_test,soup_dict_test,vege_dict_test,dessert_dict_test,애매]
    
    # 저장된 모델에서 포함된 이미 선별한 메뉴는 x에서 삭제해주기
    hoho = []
    for i in menu_all:
        for j in i.keys():
            for q in i[j]:
                hoho.append(q)
                
    for i in hoho:
        try:
            x.remove(i)
        except:
            pass
        
    # 선별작업
    cnt = 0
    for i in range(len(x)):
        cnt += 1
        if cnt == 10:
            cnt = 0
            print("자동저장 되었습니다.\n")
            with open('./menu/menu_all.pickle','wb') as fw:
                pickle.dump(menu_all, fw)
                    
        while True:
            print("{} : {}".format(i,x[i]))
            target = str(input("카테코리를 입력하세요 : "))
            if target in rice_dict_test.keys():
                rice_dict_test['{}'.format(target)].append(x[i])
                break
                
            elif target in soup_dict_test.keys():
                soup_dict_test['{}'.format(target)].append(x[i])
                break
            
            elif target in vege_dict_test.keys():
                vege_dict_test['{}'.format(target)].append(x[i])
                break
            
            elif target in dessert_dict_test.keys():
                dessert_dict_test['{}'.format(target)].append(x[i])
                break
                
            elif target in 애매.keys():
                애매['{}'.format(target)].append(x[i])
                break
            
            # 멈춰라는 명령어를 입력하면 종료가 되면서 메뉴 사전을 저장
            elif target == "멈춰":
                with open('./menu/menu_all.pickle','wb') as fw:
                    pickle.dump(menu_all, fw)
                    return menu_all
            # 잘못된 카테고리 입력 시 다시 입력하게 만들기
            else:
                print("존재하지 않는 카테고리 입니다.")
                continue
    

                    
    with open('./menu/menu_all.pickle','wb') as fw:
        pickle.dump(menu_all, fw)
    
    return menu_all, hoho

rice_dict_test = {
            "쌀밥류": list(set(list())),
            "덮밥국밥류": list(set(list())),
            "비빔밥볶음밥류": list(set(list())),
            "김초밥류": list(set(list())),
            "국수류": list(set(list())),
            "죽류" : list(set(list()))
            }
soup_dict_test = {
            "국탕류": list(set(list())),
            "찌개류": list(set(list())),
            "스프류": list(set(lisit()))
            }
vege_dict_test = {
            "김치류": list(set(list())),
            "샐러드류": list(set(list())),
            "튀김류": list(set(list())),
            "전류": list(set(list())),
            "찜류": list(set(list())),
            "볶음류": list(set(list())),
            "조림류": list(set(list())),
            "구이류": list(set(list())),
            "무침류": list(set(list())),
            "장아찌류": list(set(list()))
            }
dessert_dict_test = {
            "음료": list(set(list())),
            "유제품": list(set(list())),
            "빵과자류": list(set(list())),
            "과일류" :  list(set(list()))
            }

In [29]:
input_menu(menu)

([{'쌀밥류': ['쌀밥',
    '잡곡밥',
    '뿌리채소영양밥',
    '버섯영양밥',
    '완두콩밥',
    '수수밥',
    '귀리밥',
    '추가밥',
    '찰현미밥',
    '기장밥',
    '오곡밥',
    '흑미밥',
    '렌틸콩밥',
    '검정콩밥',
    '팥밥',
    '병아리콩밥',
    '작은밥',
    '차조밥',
    '치자밥',
    '찰보리밥',
    '보리밥',
    '단호박영양밥',
    '찹쌀밥',
    '강낭콩밥',
    '꽁보리밥',
    '옥수수밥'],
   '덮밥국밥류': ['짜장잡채밥',
    '애플카레라이스',
    '마파두부덮밥',
    '참치회덮밥',
    '돈육버섯고추장덮밥',
    '유산슬덮밥',
    '고기듬뿍카레라이스',
    '순대국밥',
    '매운순대국',
    '참치덮밥',
    '카레덮밥',
    '매운순대국밥',
    '짜장잡채덮밥',
    '카레라이스',
    '김치제육덮밥',
    '짜장밥',
    '닭살카레라이스',
    '제육김치덮밥',
    '쇠고기숙주규동덮밥',
    '치킨마요덮밥',
    '짜장덮밥',
    '소불고기덮밥',
    '인도커리',
    '불고기덮밥',
    '고추잡채덮밥',
    '잡채밥',
    '후리가케덮밥',
    '삼계국밥',
    '낙지덮밥',
    '단호박카레라이스',
    '쇠고기규동덮밥',
    '참치마요덮밥',
    '돼지국밥',
    '쇠고기덮밥',
    '연복풍덮밥',
    '유니짜장밥'],
   '비빔밥볶음밥류': ['열무보리비빔밥',
    '볶음밥',
    '해물볶음밥',
    '낙지볶음덮밥',
    '파인애플볶음밥',
    '불고기필라프',
    '쇠고기볶음밥',
    '나물비빔밥',
    '해물굴소스볶음밥',
    '삼겹살마늘볶음밥',
    '열무비빔밥',
    '날치알김치볶음밥',
    '소고기콩나물

In [30]:
def make_columns_menu(data):
    with open('./menu/menu_all.pickle', 'rb') as ma:
        menu_all_test = pickle.load(ma)
        rice_dict_test_1 = menu_all_test[0]
        soup_dict_test_1 = menu_all_test[1]
        vege_dict_test_1 = menu_all_test[2]
        dessert_dict_test_1 = menu_all_test[3]
        애매_1 = menu_all_test[4]
    
    hihi = ['중식','석식']
    
    for j in hihi:
        for i in menu_all_test:
            for k in i.keys():
                data['{}({})'.format(k,j)] = 0

In [31]:
def fill_menu(data):
    with open('./menu/menu_all.pickle', 'rb') as ma:
        menu_all_test = pickle.load(ma)
        rice_dict_test_1 = menu_all_test[0]
        soup_dict_test_1 = menu_all_test[1]
        vege_dict_test_1 = menu_all_test[2]
        dessert_dict_test_1 = menu_all_test[3]
        애매_1 = menu_all_test[4]
        
    a = data['중식메뉴'].tolist()
    b = data['석식메뉴'].tolist()

    tem2 = ['중식','석식']
    tem = [a,b]
    for w in range(2):
        for i in range(len(tem[w])):
            x = tem[w][i].split()
            for j in x:
                for k in menu_all_test:
                    for t,q in k.items():
                        if j in q:
                            data["{}({})".format(t,tem2[w])].iloc[i] += 1

In [32]:
make_columns_menu(temp_train)
make_columns_menu(temp_t)

fill_menu(temp_train)
fill_menu(temp_t)

In [33]:
temp_train.head()

,년,월,일,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,...,볶음류(석식),조림류(석식),구이류(석식),무침류(석식),장아찌류(석식),음료(석식),유제품(석식),빵과자류(석식),과일류(석식),애매(석식)
0,2016,2,1,2016-02-01,월,2601,50,150,238,0.0,...,0,1,1,1,0,0,0,0,0,0
1,2016,2,2,2016-02-02,화,2601,50,173,319,0.0,...,1,0,0,1,0,0,0,0,1,0
2,2016,2,3,2016-02-03,수,2601,56,180,111,0.0,...,1,0,1,0,0,0,0,0,0,0
3,2016,2,4,2016-02-04,목,2601,104,220,355,0.0,...,0,0,1,0,1,0,0,0,0,0
4,2016,2,5,2016-02-05,금,2601,278,181,34,0.0,...,1,0,1,1,0,0,0,0,0,0


In [34]:
temp_t.head()

,년,월,일,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,...,볶음류(석식),조림류(석식),구이류(석식),무침류(석식),장아찌류(석식),음료(석식),유제품(석식),빵과자류(석식),과일류(석식),애매(석식)
0,2021,1,27,2021-01-27,수,2983,88,182,5,358.0,...,2,0,0,0,1,0,0,0,0,0
1,2021,1,28,2021-01-28,목,2983,104,212,409,348.0,...,0,0,0,2,0,0,0,0,0,0
2,2021,1,29,2021-01-29,금,2983,270,249,0,294.0,...,0,0,1,1,0,0,0,0,0,0
3,2021,2,1,2021-02-01,월,2924,108,154,538,322.0,...,0,1,0,2,0,0,0,0,0,0
4,2021,2,2,2021-02-02,화,2924,62,186,455,314.0,...,1,0,1,1,0,0,0,0,0,0
